In [0]:
import numpy as np
import pandas as pd

In [0]:
train = pd.read_csv('data/train.csv', index_col='id')

In [ ]:
# RobustScaler
from sklearn.preprocessing import RobustScaler
train_for_scaling = train.iloc[:,2:] #type과 fiberID 제외하고 데이터 생성
robustScaler = RobustScaler()
print(robustScaler.fit(train_for_scaling))
train_robustscaled = robustScaler.transform(train_for_scaling) #robustscaling 적용
train_robustscaled = pd.DataFrame(train_robustscaled,columns=train_for_scaling.columns)
train_robustscaled = pd.concat([train[['type', 'fiberID']], train_robustscaled], axis=1) 
train_robustscaled #robustscaling과 type,fiberID 합치기

In [0]:
from sklearn.model_selection import train_test_split
X = train_robustscaled.drop('type', axis = 1)
y = train_robustscaled['type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

In [52]:
#1. Randomforest
from sklearn.ensemble import RandomForestClassifier
rnd_model = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=True, n_jobs=-1)
rnd_model.fit(X_train, y_train)
rnd_model.score(X_test,y_test)

0.7486437160929024

In [65]:
#2. XgBoost 
import xgboost as xgb
xgb_model = xgb.XGBClassifier(n_estimators=1000,
                        n_jobs=4,
                        max_depth=15,
                        learning_rate=0.05,
                        gamma = 0.02,
                        subsample = 0.9,
                        colsample_bytree=0.9,
                        missing=-999,
                        tree_method='gpu_hist')
xgb_model.fit(X_train,y_train)
xgb_model.score(X_test,y_test)

0.8775219380484512

In [1]:
#3. LGBM
from lightgbm import LGBMRegressor
lgbm_model = LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=10,
                                learning_rate=0.1, n_estimators=2000, max_depth=15,
                                bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)
lgbm_model.fit(X_train,y_train)
lgbm_model.score(X_test,y_test)

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


NameError: name 'LGBMClassifier' is not defined

In [2]:
#GridSearch
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier(objective='binary')

param_dist = {'boosting_type':['rf'],
              'n_estimators': [1500,2000,2500],
              'max_depth': [15,30,45],
              'learning_rate': [0.01,0.05,0.1],
              'bagging_freq': [1],
              'bagging_fraction':[0.9]
             }
grid_search = GridSearchCV(lgbm_model, n_jobs=6, param_grid=param_dist, cv=5, scoring="neg_log_loss", verbose=5)
grid_search.fit(X,y)

In [ ]:
#parameter 찾기
grid_search.best_estimator_

In [ ]:
#Test 데이터 불러와서 index 설정
test = pd.read_csv('data/test.csv')
test = test.reset_index()
id_for_index = test['id']

In [ ]:
#Test 데이터 결과값 예측
test_pred_prob_lgbm_robustscaled = lgbm_model.predict_proba(test)
test_pred_prob_lgbm_robustscaled

In [ ]:
# Dataframe 형식으로 변환
result = pd.DataFrame(data=test_pred_prob_xgb_robustscaled, index=id_for_index, columns=lgbm_model.classes_)
result.head()

In [ ]:
#Sample Submission 형식으로 변환
sample = pd.read_csv("/data/sample_submission.csv")
col_order = sample.columns[1:]
result = result[col_order]
result

In [ ]:
#csv 내보내기
result.to_csv('/content/drive/My Drive/DACON/data/1st_submission_rb_lgbm.csv',sep=',')